In [16]:
from IPython.display import Image, display
from ase.data.colors import jmol_colors
from ase.visualize import view
import subprocess, os
from ase.io import read, write, Trajectory
import asap3
import numpy as np
from ase.io.pov import get_bondpairs

rotation = '-90x,90y,0z'
#rotation = '-80x,0y,0z'
#rotation = '0x,0y,0z'
# (x1, y1, x2, y2), x1, y1 is the position of left-bottom corner, x2, y2 is the position of right-up corner
bbox = (-15, -15, 15, 15)
generic_settings = {
    'rotation'       : rotation, # text string with rotation (default='' )
    'bbox'           : bbox,
    'radii'          : .95, # float, or a list with one float per atom
    'show_unit_cell' : 1,   # 0, 1, or 2 to not show, show, and show all of cell
#    'colors'         : colors,
#    'scale'          : scale,
#    'extra_offset'   : (0., 0.)
}

povray_settings = {
#    'run_povray'     : False, # Run povray or just write .pov + .ini files
    'display'        : False,# Display while rendering
    'pause'          : True, # Pause when done rendering (only if display)
    'transparent'    : False,# Transparent background
    'canvas_width'   : 1000, # Width of canvas in pixels
    'canvas_height'  : None, # Height of canvas in pixels
    'camera_dist'    : 50.,  # Distance from camera to front atom
    'image_plane'    : None, # Distance from front atom to image plane
    'camera_type'    : 'orthographic', #'orthographic', #'perspective', # perspective, ultra_wide_angle
    'point_lights'   : [],             # [[loc1, color1], [loc2, color2],...]
    'area_light'     : [(2., 3., 40.), # location
                        'White',       # color
                        .7, .7, 3, 3], # width, height, Nlamps_x, Nlamps_y
    'background'     : 'White',        # color
    'celllinewidth'  : 0.0,  # Radius of the cylinders representing the cell
    'textures'       : None, 
    'transmittances' : None,
    'bondlinewidth'  : 0.1,
    'bondatoms'      : [],
}

def plot_atoms(plt_atoms, name, render=True, remove_pov=True):
    atoms = plt_atoms.copy()
    atoms.translate((1.5, -1.5, 0))
    atoms.wrap()
    
    bond_pairs = []
    O2_idx = []
    water_idx = []
    hydro_idx = []
    O1_idx = []
    
    O_idx = np.where(atoms.get_atomic_numbers() == 8)[0]
    H_idx = np.where(atoms.get_atomic_numbers() == 1)[0]
    # deal with periodic image problem
    nl =  asap3.FullNeighborList(2, atoms)
    for i in O_idx:
        n_idx, n_diff, _ = nl.get_neighbors(i)
        n_dist = np.linalg.norm(n_diff, axis=1)
        
        # get bond pairs, water indices, OH indices, O2 indices
        n_idx_idx = [x for x in np.where((n_dist < 1.23) & (n_dist > 0.1))[0] if atoms.numbers[n_idx[x]] == 1]
        OH_idx = n_idx[n_idx_idx]
        
        # deal with periodic image problem
        rel_diff = atoms.positions[OH_idx] - atoms.positions[i]
        atoms.positions[OH_idx] -= rel_diff - n_diff[n_idx_idx]
        OO_idx = [n_idx[x] for x in np.where((n_dist < 1.9) & (n_dist > 0.1))[0] if atoms.numbers[n_idx[x]] == 8]
        if OH_idx.size > 0:
            for idx in OH_idx:
                bond_pairs.append([i, idx, (0, 0, 0), 1])
            if len(OH_idx) == 2:
                water_idx.append((i, OH_idx[0], OH_idx[1]))
            if len(OH_idx) == 1:
                hydro_idx.append((i, OH_idx[0]))
        elif len(OH_idx) == 0: 
            O1_idx.append(i)
        if len(OO_idx) == 1:
            O2_idx.append((i, OO_idx[0]))
            
    for o2 in O2_idx:
        bond_pairs.append([o2[0], o2[1], (0, 0, 0), 2, (0.2, 0.2, 0)])

    O2_idx = np.asarray(O2_idx)
    water_idx = np.asarray(water_idx)
    hydro_idx = np.asarray(hydro_idx)
    O1_idx = np.asarray(O1_idx)
    
    # control radii
    Z = atoms.get_atomic_numbers()
    colors = jmol_colors[Z]
    radii = np.ones_like(atoms.get_atomic_numbers(), dtype=float)
    transmit = np.zeros_like(atoms.get_atomic_numbers(), dtype=float)
    radii += 0.3
    radii[O_idx] = 0.1
    radii[H_idx] = 0.1
    if O2_idx.size > 0:
        radii[O2_idx.reshape(-1)] = .5
    # radii[water_idx[:, 0]] = .5
    # radii[water_idx[:, 1:].reshape(-1)] = .3
#     surf_water = water_idx[atoms.positions[:,2][water_idx[:, 0]] < 13.5]
#     radii[surf_water[:, 0]] = .5
#     radii[surf_water[:, 1]] = .3
#     radii[surf_water[:, 2]] = .3
    if O1_idx.size > 0:
        radii[O1_idx] = 0.5
    
#    radii[87] = .5
    textures = ['jmol'] * len(atoms)
    transmit[water_idx.reshape(-1)] = 0
    if hydro_idx.size > 0:
        radii[hydro_idx[:, 0]] = .5
        radii[hydro_idx[:, 1]] = .3
        colors[hydro_idx[:, 0]] = [0, 0, 1]
        
    # plot things
    ase_settings = generic_settings.copy()
    pov_settings = povray_settings.copy()

    ase_settings['colors'] = colors
    ase_settings['radii'] = radii
    pov_settings['bondatoms'] = bond_pairs
    pov_settings['transmittances'] = transmit
    pov_settings['textures'] = textures
    write('{}.pov'.format(name), atoms, povray_settings=pov_settings, **ase_settings)
    if render:
        pov_run = subprocess.run('povray {}.ini'.format(name), 
                                 shell=True, 
                                 stdout=subprocess.DEVNULL, 
                                 stderr=subprocess.DEVNULL)
    if remove_pov:
        os.remove('{}.ini'.format(name))
        os.remove('{}.pov'.format(name))

    # return water_idx

In [4]:
atoms = read("../../aspirin_dft.traj", 150000)

In [19]:
view(atoms)

<Popen: returncode: None args: ['/home/xinyang/miniconda3/bin/python', '-m',...>

In [17]:
bond_pairs = get_bondpairs(atoms, radius=1.1)
ase_settings = generic_settings.copy()
pov_settings = povray_settings.copy()

pov_settings["bondatoms"] = bond_pairs
write('test.pov', atoms, povray_settings=pov_settings, **ase_settings)

In [5]:
view(atoms)

<Popen: returncode: None args: ['/home/xinyang/miniconda3/bin/python', '-m',...>

In [ ]:
def find_bonds(atoms):
    